In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [2]:
df=pd.read_csv('CKD.csv')
df=pd.get_dummies(df,dtype=int,drop_first=True)
df

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [3]:
df.isnull().values.any()

np.False_

In [4]:
indep=df[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=df['classification_yes']

In [5]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(indep)

DECISION_TREE

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
param_grid={'criterion':['log_loss', 'entropy', 'gini'],
              'max_features': [None,'sqrt','log2'],
              'max_depth': [None, 10, 20],  # Use max_depth instead of max_features
    'min_samples_split': [2, 10]}
grid=GridSearchCV(DecisionTreeClassifier(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(indep,dep)



Fitting 5 folds for each of 54 candidates, totalling 270 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['log_loss', 'entropy', 'gini'],
                         'max_depth': [None, 10, 20],
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_split': [2, 10]},
             scoring='f1_weighted', verbose=3)

In [15]:
re=grid.cv_results_
y_pred=grid.predict(indep)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(dep,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(dep,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(dep,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_score=roc_auc_score(dep,grid.predict_proba(indep)[:,1])
print("The roc_score:",roc_score)

[[150   0]
 [  0 249]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       150
           1       1.00      1.00      1.00       249

    accuracy                           1.00       399
   macro avg       1.00      1.00      1.00       399
weighted avg       1.00      1.00      1.00       399

The f1_macro value for the best parameters {'criterion': 'entropy', 'max_depth': None, 'max_features': 'log2', 'min_samples_split': 2}: 1.0
The roc_score: 1.0


In [16]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_max_features,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.020802,0.023252,0.052180,0.042511,log_loss,None,None,2,"{'criterion': 'log_loss', 'max_depth': None, '...",0.452683,0.962813,0.936147,0.987539,1.000000,0.867836,0.208727,34
1,0.007188,0.001361,0.005410,0.001084,log_loss,None,None,10,"{'criterion': 'log_loss', 'max_depth': None, '...",0.452683,0.938022,0.936147,0.987539,1.000000,0.862878,0.206697,46
2,0.004879,0.000741,0.018141,0.022172,log_loss,None,sqrt,2,"{'criterion': 'log_loss', 'max_depth': None, '...",0.405000,0.913778,0.962618,0.975000,0.868457,0.824971,0.213358,54
3,0.019000,0.028783,0.011055,0.003834,log_loss,None,sqrt,10,"{'criterion': 'log_loss', 'max_depth': None, '...",0.876918,0.975000,0.962618,0.962618,0.974684,0.950368,0.037129,19
4,0.005801,0.000796,0.005292,0.000249,log_loss,None,log2,2,"{'criterion': 'log_loss', 'max_depth': None, '...",0.938022,0.975148,0.936147,0.987539,0.987380,0.964847,0.023117,6
5,0.004230,0.000378,0.005068,0.000819,log_loss,None,log2,10,"{'criterion': 'log_loss', 'max_depth': None, '...",0.887854,0.975000,0.975000,0.987539,0.741005,0.913280,0.093229,29
6,0.005843,0.000609,0.004388,0.000489,log_loss,10,None,2,"{'criterion': 'log_loss', 'max_depth': 10, 'ma...",0.452683,0.950521,0.936147,0.987539,1.000000,0.865378,0.207664,40
7,0.005936,0.000544,0.028533,0.030695,log_loss,10,None,10,"{'criterion': 'log_loss', 'max_depth': 10, 'ma...",0.452683,0.938022,0.936147,0.987539,1.000000,0.862878,0.206697,46
8,0.020180,0.030841,0.004687,0.000586,log_loss,10,sqrt,2,"{'criterion': 'log_loss', 'max_depth': 10, 'ma...",0.950296,0.962618,0.922956,1.000000,0.987298,0.964634,0.027244,7
9,0.004705,0.000979,0.037874,0.039457,log_loss,10,sqrt,10,"{'criterion': 'log_loss', 'max_depth': 10, 'ma...",0.937280,0.901354,0.949628,0.962813,0.987380,0.947691,0.028507,23
